In [1]:
import syft
import syft.nn as nn
from syft.controller import tensors, models
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)

import numpy as np
from syft import FloatTensor
import torch
from torch.autograd import Variable

In [2]:
input = FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]], autograd=True)
target = FloatTensor([[0],[0],[1],[1]]).autograd(True)
grad = FloatTensor([[1],[1],[1],[1]]).autograd(False)

np.random.seed(1)

weights1 = FloatTensor(np.random.rand(3,4)).autograd(True)
weights2_data = np.array([0.2044522,0.8781174,-0.02738759,-0.6704675])
weights2_data.shape = (4,1)
weights2 = FloatTensor(weights2_data).autograd(True)

input_t = Variable(torch.FloatTensor(input.to_numpy()), requires_grad=True)
target_t = Variable(torch.FloatTensor(target.to_numpy()), requires_grad=True)
weights1_t = Variable(torch.FloatTensor(weights1.to_numpy()), requires_grad=True)
weights2_t = Variable(torch.FloatTensor(weights2.to_numpy()), requires_grad=True)

grad_t = Variable(torch.FloatTensor(grad.to_numpy()))

In [3]:
weights2

[[ 0.2044522 ]
 [ 0.8781174 ]
 [-0.02738759]
 [-0.6704675 ]]
[syft.FloatTensor:5 grad:None size:4x1 c:[] p:[] init:]

In [4]:
weights1_t.grad = None
weights2_t.grad = None
target_t.grad = None
input_t.grad = None

layer_1_t = input_t.mm(weights1_t).clamp(min=0)
layer_2_t = layer_1_t.mm(weights2_t).clamp(min=0)

print("Layer2 before relu:")
print(layer_1_t.mm(weights2_t))
print("Layer2 after relu:")
print(layer_2_t)

diff_t = layer_2_t - target_t
loss_t = diff_t ** 2
loss_t.backward(grad_t)

#print(loss_t.sum().data[0])

weights1_t.data -= weights1_t.grad.data
weights2_t.data -= weights2_t.grad.data
weights1_t.grad.data

Layer2 before relu:
Variable containing:
 0.0834
-0.0423
 0.5984
 0.4727
[torch.FloatTensor of size 4x1]

Layer2 after relu:
Variable containing:
 0.0834
 0.0000
 0.5984
 0.4727
[torch.FloatTensor of size 4x1]




-0.3798 -1.6312  0.0509  1.2455
-0.2156 -0.9260  0.0289  0.7070
-0.3457 -1.4848  0.0463  1.1337
[torch.FloatTensor of size 3x4]

In [5]:
weights2_t.grad.data


-1.6003
-2.3465
-0.9054
-2.0846
[torch.FloatTensor of size 4x1]

In [6]:
layer_1 = input.mm(weights1).relu()
layer_2 = layer_1.mm(weights2).relu()

print("Layer2 before relu:")
print(layer_1.mm(weights2))
print("Layer2 after relu:")
print(layer_2)

diff = (layer_2 - target)
loss = diff ** 2 # Mean Squared Error Loss

loss.backward(grad)
weights1 -= weights1.grad()
weights2 -= weights2.grad()

Layer2 before relu:
   0.0833662  
  -0.04233359 
   0.5984493  
   0.4727495   

Layer2 after relu:
   0.0833662 
   0.        
   0.5984493 
   0.4727495  



In [7]:
weights1.grad()

[[-0.3797909  -1.631193    0.05087525  1.245462  ]
 [-0.2155951  -0.9259757   0.02888024  0.7070087 ]
 [-0.3457021  -1.484782    0.04630886  1.133673  ]]
[syft.FloatTensor:29 grad:None size:3x4 c:[] p:[27, 25] init:mm]

	-----------creators-----------
	[syft.FloatTensor:27 grad:None size:3x4 c:[29] p:[1] init:transpose]
	[syft.FloatTensor:25 grad:None size:4x4 c:[28, 29] p:[24, 22] init:mul_elem]
	------------------------------



In [8]:
weights2.grad()

[[-1.600298 ]
 [-2.346517 ]
 [-0.9054275]
 [-2.084625 ]]
[syft.FloatTensor:30 grad:None size:4x1 c:[] p:[21, 19] init:mm]

	-----------creators-----------
	[syft.FloatTensor:21 grad:None size:4x4 c:[30] p:[7] init:transpose]
	[syft.FloatTensor:19 grad:None size:4x1 c:[22, 30] p:[18, 15] init:mul_elem]
	------------------------------

